In [44]:
!pip3 install lxml
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Dr. Mattei, Tulane University. SimpleText Notebook. 2020. 
Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

In [45]:
#Import CSV containing mass amount of scraped reviews for analysis
reviewsByGender = pd.read_csv("./revs_gendered.csv")
reviewsByGender.dropna(inplace = True)
reviewsByGender

,Unnamed: 0,Review,Gender
0,0,Very good,Male
1,1,"Exelent he really care about you, he is very p...",Male
2,2,Dr. A-Rahim was very knowledgeable about my lo...,Male
3,3,Horrible physician treats you like a kid. Pts ...,Female
4,4,Dr Aaberg has been a very good dr for my husba...,Male
...,...,...,...
8890,8890,Very helpful and sincere about my pain needs! ...,Male
8891,8891,"Several of my children see Dr. Zach, she is a ...",Female
8892,8892,Amazing Pediatric Neurologist stays up to date...,Female
8893,8893,Dr. Zach has been a Godsend for our daughter! ...,Female


In [46]:
# Vectorize the whole thing...
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Vectorize and play with token sizes...
vec = TfidfVectorizer(min_df = 2, 
                      max_df = 0.98, 
                      ngram_range=(1,1)) # play with min_df and max_df

reviews = reviewsByGender['Review']

# transform this into a sparse vector!
vec.fit(reviews)
tf_idf_sparse = vec.transform(reviews)
tf_idf_sparse

#only 222 unique words due to min_df 0.03 --> change to 2, maybe change to 5 (could get meaningless words)

<8895x7365 sparse matrix of type '<class 'numpy.float64'>'
	with 322903 stored elements in Compressed Sparse Row format>

In [47]:
revs = dict(zip(reviewsByGender.Review, reviewsByGender.Gender))

In [48]:
#get list of cleaned, stemmed adjectives, for later analysis. 
vect = CountVectorizer()
vect.fit(revs)
words = vect.get_feature_names()

adjectives = []
stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

In [49]:
print(tf_idf_sparse[0, :])

  (0, 7004)	0.5113918885739566
  (0, 2832)	0.8593476225025365


In [50]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    reviewsByGender['Gender'], 
                                                    test_size=0.2)

In [51]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

/Users/laurensussman/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [52]:
logisticRegr.classes_

array(['Female', 'Male'], dtype=object)

In [53]:
logisticRegr.coef_[:,0]

array([0.20222314])

In [54]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [55]:
df_weights.sort_values('weight', ascending=False)[:25]

,word,weight
3,he,13.571158
85,his,8.862567
84,him,7.275667
328,surgery,3.000313
87,staff,1.651830
410,pain,1.640181
920,man,1.626908
86,outstanding,1.603044
14,was,1.441894
1358,mother,1.376205


In [56]:
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
86,outstanding,1.603044
1111,arrogant,1.075631
302,medical,1.075210
1,good,1.060960
944,positive,0.950070
2291,kindness,0.938643
4753,faber,0.931922
765,free,0.924356
678,respectful,0.777541
356,friendly,0.764605


In [57]:
df_weights.sort_values('weight', ascending=True)[:25]

,word,weight
44,she,-13.909459
396,her,-10.995458
4699,ma,-2.677467
5981,kabbash,-1.901633
588,woman,-1.690494
4442,eady,-1.679180
7309,yablonski,-1.412029
2376,breast,-1.312861
6708,pabolu,-1.227939
77,not,-1.177212


In [58]:
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
#maybe take out adjective specification
#vocabulary being pruned down too aggressively
femaleAdjectives

,word,weight
4442,eady,-1.679180
2633,pediatrician,-1.091084
3512,cabalona,-1.052787
691,primary,-1.031698
15,knowledgeable,-0.880945
712,happy,-0.858413
303,unprofessional,-0.748575
2600,lowest,-0.730325
481,wonderful,-0.729376
699,smart,-0.712585


In [59]:
#verify results
agentic_terms = ['assertive','confident','aggressive','ambitious','dominan','forceful','independent','daring','outspoken','intellectua','earn','gain','know','insight','think']
communal_terms = ['sympathetic','kind','help','affection','sensitive','nurtur','agreeab','tactful','interpersonal','warm','car','tactful']
socio_communal = ['husband','wife','kid','babies','brother','child','colleague','family']

print('Results for most commonly found female adjectives:')
print('Agentic: ', femaleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', femaleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', femaleAdjectives['word'].isin(socio_communal).sum())
print('')
print('Results for most commonly found male adjectives:')
print('Agentic: ', maleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', maleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', maleAdjectives['word'].isin(socio_communal).sum())

Results for most commonly found female adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0

Results for most commonly found male adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0


In [60]:
#can use medical reviews to run with recommnedation letter reviews to predict which is which, then throw out 
#heavily medical terms

# recommendation system
# maybe seek out positive words that don't distinguish between male and female
# 4-class problem rather than two-class problem: take in star data and add pos/neg dimension to the data
# word vector stuff from NLP class: use language models and figure out how often it appears in male context vs
# female context. Ex: look at similarities between words: compare "leadership" to "man/men" 

#doubt raisers may not come out from this analysis: "she may not be the best in her class..."
# subtle, hard to catch -> maybe the word "not"
# append "not" to nearest adjective, negated adjectives might indicate doubt raisers
# hedging terms to weaken a statement (weasel words): refusing to take a stand/hesitant
# "somewhat", "mostly"

# how to rephrase it: female-correlated to neutral, remove hedging terms
# take an education based approach: "this can be conveyed as ___. If that is not your intention, consider ___"
# crowdsourcing: emphasize anonymization

## How well does our healthgrades data predict the gender of our reference letter subjects?

In [61]:
letters = pd.read_csv("./letters_tidied.csv")
letters.dropna(inplace = True)
letters.columns = ['Letter', 'Gender']


# testing data:
letters

,Letter,Gender
0,"dear review committee members,\r\n\r\nit is wi...",0.0
1,this letter addresses some of my thoughts and ...,0.0
2,I am writing this letter to recommend REDACTED...,1.0
4,it is my sincere pleasure to nominate dr. REDA...,0.0
5,i am writing to strongly recommend REDACTED RE...,1.0
...,...,...
92,I am writing to highly recommend that you acce...,0.0
93,i am writing to highly recommend that you acce...,0.0
94,i am writing to highly recommend that you acce...,0.0
95,\r\ni am writing to highly recommend that you ...,0.0


In [62]:
# Training data:

reviewsByGender['Gender'] = reviewsByGender['Gender'].replace('Female', 0.0)
reviewsByGender['Gender'] = reviewsByGender['Gender'].replace('Male', 1.0)
reviewsByGender.dropna(inplace = True)
reviewsByGender


,Unnamed: 0,Review,Gender
0,0,Very good,1.0
1,1,"Exelent he really care about you, he is very p...",1.0
2,2,Dr. A-Rahim was very knowledgeable about my lo...,1.0
3,3,Horrible physician treats you like a kid. Pts ...,0.0
4,4,Dr Aaberg has been a very good dr for my husba...,1.0
...,...,...,...
8890,8890,Very helpful and sincere about my pain needs! ...,1.0
8891,8891,"Several of my children see Dr. Zach, she is a ...",0.0
8892,8892,Amazing Pediatric Neurologist stays up to date...,0.0
8893,8893,Dr. Zach has been a Godsend for our daughter! ...,0.0


In [63]:
x_train = reviewsByGender['Review']
y_train = reviewsByGender['Gender']
x_new = letters['Letter']

In [64]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

vec = CountVectorizer(max_df=100, min_df=2, binary=False, strip_accents = None)
X_train = vec.fit_transform(x_train)
X_test = vec.transform(x_new)

In [65]:
nb = BernoulliNB()
nb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [69]:
gender_pred = nb.predict(X_test)
print(classification_report(letters['Gender'], gender_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.69      0.97      0.81        61
           1       0.50      0.07      0.12        28

    accuracy                           0.69        89
   macro avg       0.60      0.52      0.47        89
weighted avg       0.63      0.69      0.59        89



In [70]:
lr = LogisticRegression(C=1e25, class_weight="balanced")
lr.fit(X_train, reviewsByGender['Gender'])
gender_pred = lr.predict(X_test)
print(classification_report(letters['Gender'], gender_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.69      0.15      0.24        61
           1       0.32      0.86      0.46        28

    accuracy                           0.37        89
   macro avg       0.50      0.50      0.35        89
weighted avg       0.57      0.37      0.31        89



### This means: Logistic regression predicts accurate gender of letters at 46%
### Naive Bayes predicts accurate gender at 69%

Super small sample size though, so results are not definite